In [1]:
from src.models.Encodec import Encodec
encodec = Encodec()


/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
from audiocraft.modules.codebooks_patterns import DelayedPatternProvider

pattern_provider = DelayedPatternProvider(4)

import torch
test = torch.randn(1,24000*20)

test2 = {"wav" : torch.randn(2,1,24000*20),
        "original_lens" : torch.Tensor([[12000],[12000]]).int()}

codes = encodec(test)


In [3]:
test2['original_lens'].shape

torch.Size([2, 1])

In [4]:
for i,l in enumerate(test2['original_lens']):
    print(l)

tensor([12000], dtype=torch.int32)
tensor([12000], dtype=torch.int32)


In [5]:
codes.shape

torch.Size([1, 4, 1500])

In [6]:
pattern = pattern_provider.get_pattern(codes.shape[-1])

In [7]:
from src.models.model import MuMRVQ
from src.models.Transformer.Encoder import VanillaEncoder
from src.models.Transformer.Decoder import VanillaDecoder
model = MuMRVQ(VanillaEncoder(layers=2), VanillaDecoder(layers=2) ,encodec, debug = True, pattern="delay")

/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [8]:
out_ = model(test2)

codes: torch.Size([2, 4, 1504])
padding_mask: torch.Size([2, 1505])

codes: torch.Size([2, 4, 1024])
padding_mask: torch.Size([2, 1025])
torch.Size([2, 1024, 960])
torch.Size([2, 1025, 960])


: 

: 

In [ ]:
out_['logits'][:,:,1:,:].shape

torch.Size([2, 1027, 3, 1505])

In [ ]:
out_['codes'].shape

torch.Size([2, 4, 1504])

In [ ]:
from torch.nn import CrossEntropyLoss

CrossEntropyLoss(ignore_index=1026)(out_['logits'][:,:,:,1:],out_['codes'])

tensor(7.2012, grad_fn=<NllLoss2DBackward0>)

In [ ]:
out_['codes']

tensor([[[1024,  732,  732,  ..., 1026, 1026, 1026],
         [1024, 1024,  348,  ..., 1026, 1026, 1026],
         [1024, 1024, 1024,  ..., 1026, 1026, 1026],
         [1024, 1024, 1024,  ..., 1026, 1026, 1026]],

        [[1024,  732,  732,  ..., 1026, 1026, 1026],
         [1024, 1024,  691,  ..., 1026, 1026, 1026],
         [1024, 1024, 1024,  ..., 1026, 1026, 1026],
         [1024, 1024, 1024,  ..., 1026, 1026, 1026]]])

In [ ]:
test = torch.softmax(torch.Tensor([1,-float('inf')]),dim=0)